# Subprocess
Python version: 3.6

* subprocess模块用于孵化新进程，与input/output/error管道连接，返回return code。
* subprocess模块用于替换一些老的模块：os.system, os.spawn*

## Subprocess的使用
比较建议的调用subprocess的方式是使用run()功能，对于高阶的应用，可以直接使用Popen接口。

### run的使用
* run中包含args参数，用于启动process，这个参数可以是一个list，或一个string，针对这个list或string做了以下测试。发现命令如果带参数，中间由空格，则需要以list来完成。如果只是一个命令，比如“ls”,则直接使用string即可。
*不知道是否可以设置一个长命令也用string表示，该如何表示？*（参照Popen下的解释和解决方案）
* run返回一个CompltedProcess实例
* 默认不会捕获stdin以及stdout，如果需要，应传递PIPE给stdout以及stderr
subproces.PIPE是一个特殊的值，通常与Popen.communicate()一起使用
* timeout参数，会被传递给Popen.communicate().如果超时，子进程将会被kill并且等待。在子进程被终止后，意外处理会调用TimeoutExpired.
* input参数，会被传递给Popen.communicate()而到达subprocess的stdin。如果使用input，在encoding或者error被指定以及universal_newlines被设置为true的情况下，input必须是一个字节流或者是一个字符串。当这个参数被使用的时候，Popen对象创建的时候会自动使用stdin=PIPE。
* check参数，如果check为True，在process的exit code非0时，会调用CalledProcessError以外处理。
* env如果不是None，这个会被用来当作新的process的环境变量,即用来设置子进程的环境变量

In [22]:
#Examples

import subprocess

#subprocess.run('systemctl status nfs-server')
subprocess.run(["systemctl", "status", "nfs-server"])
subprocess.run('ls')
subprocess.run(["ls", "-l"])


CompletedProcess(args=['ls', '-l'], returncode=0)

In [30]:
subprocess.run("ls", input="/etc/fstab")

TypeError: a bytes-like object is required, not 'str'

In [23]:
subprocess.run(["systemctl", "status", "nfs-server"], stdout=subprocess.PIPE, stdin=subprocess.PIPE)

CompletedProcess(args=['systemctl', 'status', 'nfs-server'], returncode=0, stdout=b'\xe2\x97\x8f nfs-server.service - NFS server and services\n   Loaded: loaded (/usr/lib/systemd/system/nfs-server.service; disabled; vendor preset: disabled)\n  Drop-In: /run/systemd/generator/nfs-server.service.d\n           \xe2\x94\x94\xe2\x94\x80order-with-mounts.conf\n   Active: active (exited) since Wed 2020-03-18 15:42:53 CST; 1h 4min ago\n  Process: 15747 ExecStopPost=/usr/sbin/exportfs -f (code=exited, status=0/SUCCESS)\n  Process: 15745 ExecStopPost=/usr/sbin/exportfs -au (code=exited, status=0/SUCCESS)\n  Process: 15744 ExecStop=/usr/sbin/rpc.nfsd 0 (code=exited, status=0/SUCCESS)\n  Process: 15771 ExecStart=/bin/sh -c if systemctl -q is-active gssproxy; then systemctl reload gssproxy ; fi (code=exited, status=0/SUCCESS)\n  Process: 15758 ExecStart=/usr/sbin/rpc.nfsd (code=exited, status=0/SUCCESS)\n  Process: 15757 ExecStartPre=/usr/sbin/exportfs -r (code=exited, status=0/SUCCESS)\n Main PID:

In [28]:
subprocess.run(["systemctl", "startp", "nfs-server"], check=True, stdout=subprocess.PIPE)

CalledProcessError: Command '['systemctl', 'startp', 'nfs-server']' returned non-zero exit status 1.

## Popen构造器
Popen中的args参数仍然一样。在POSIX中，如果args是一个string，整个string都会被看作可执行程序，看作一个整体的命令，比如"systemctl status nfs-server"，系统会直接找符合这一串的可执行命令，因此会返回“No such file or directory”的错误。
*解决方法*：将shell命令分割成一系列参数，并形成一个list，可使用shlex.split()，使用方法如下：

In [14]:
import subprocess
import shlex
#测试
command_line = shlex.split("systemctl restart nfs-server")
print(command_line)

#测试直接使用这个命令作为args参数
subprocess.Popen(command_line)

subprocess.run(command_line)

['systemctl', 'restart', 'nfs-server']


CompletedProcess(args=['systemctl', 'restart', 'nfs-server'], returncode=0)